In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QVBoxLayout, QWidget, QFileDialog, QMessageBox
from PyQt5.QtGui import QPixmap, QImage
import cv2
import numpy as np
import os
from PIL import Image
import time
from split_bregman import split_bregman
from ADMM import ADMM
from chambolle_pd import tv_denoise_chambolle
from FGP import FGP
from ADMM_color import ADMM_3D
from ROF_proposed import ROFtv
#from asd_algorithm import asd_denoise
#from fgh_algorithm import fgh_denoise

# The main application class
class ImageDenoisingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Image Denoising App")
        self.image_path = None
        self.algorithm = None
        self.process_flag = None
        self.N = 100 # maxiter
        self.weight = 10 # 1/lambda
        self.iter_num = 0 # number of iteration to reach convergence
        # Create the central widget and layout
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # Add a label to display the image
        self.image_label = QLabel(self)
        self.layout.addWidget(self.image_label)

        # Add a combo box to select the image type
        self.image_type_combo = QComboBox(self)
        self.image_type_combo.addItem("B&W")
        self.image_type_combo.addItem("Colour")
        self.image_type_combo.currentIndexChanged.connect(self.image_type_changed)
        self.layout.addWidget(self.image_type_combo)

        # Add a combo box to select the algorithm
        self.algorithm_combo = QComboBox(self)
        self.layout.addWidget(self.algorithm_combo)

        # Add a button to upload the image
        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.clicked.connect(self.upload_image)
        self.layout.addWidget(self.upload_button)
        
        #TODO: 图片维度与选择不符，预警

        # Add a button to process the image
        self.process_button = QPushButton("Process Image", self)
        self.process_button.clicked.connect(self.process_image)
        # self.process_button.clicked.connect(self.plot_image)
        self.layout.addWidget(self.process_button)

        # Add a button to show image denoising dynamics
        self.dynamics_button = QPushButton("Processing Dynamics", self)
        # self.process_button.clicked.connect(self.process_image)
        self.dynamics_button.clicked.connect(self.plot_image)
        self.layout.addWidget(self.dynamics_button)

        # Add a button to download the image
        self.download_button = QPushButton("Download Image", self)
        self.download_button.clicked.connect(self.download_image)
        self.layout.addWidget(self.download_button)

        # Update the algorithm options based on the selected image type
        self.update_algorithm_options("B&W")

    # Update the algorithm options when the image type changes
    def image_type_changed(self, index):
        image_type = self.image_type_combo.itemText(index)
        self.update_algorithm_options(image_type)

    # Update the algorithm options based on the selected image type
    def update_algorithm_options(self, image_type):
        self.algorithm_combo.clear()
        if image_type == "B&W":
            self.algorithm_combo.addItem("split_bregman")
            self.algorithm_combo.addItem("ADMM")
            self.algorithm_combo.addItem("PD")
            self.algorithm_combo.addItem("FGP")
            #self.algorithm_combo.addItem("ROF")
        else:
            self.algorithm_combo.addItem("FGP")
            #self.algorithm_combo.addItem("PD")
            #self.algorithm_combo.addItem("ADMM")

    # Open a file dialog to upload an image
    def upload_image(self):
        file_dialog = QFileDialog(self)
        self.image_path, _ = file_dialog.getOpenFileName(self, "Upload Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if self.image_path:
            # s = denoised_image.shape
            pixmap = QPixmap(self.image_path)
            s = pixmap.size()
            alpha = s.width()/s.height()
            height_const = 400
            self.image_label.setPixmap(pixmap.scaled(int(alpha*height_const), int(height_const)))

    # Process the uploaded image using the selected algorithm
    def process_image(self):
        self.process_flag = None
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "Please upload an image first.")
            return None
        if self.image_type_combo.currentText() == "B&W":
            image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
            self.algorithm = self.algorithm_combo.currentText()
            if self.algorithm == "split_bregman":
                denoised_image, self.iter_num = split_bregman(image, self.weight, self.N)
            elif self.algorithm == "ADMM":
                denoised_image, self.iter_num = ADMM(image, self.weight, self.N)
                self.process_flag = True # processing finished
            elif self.algorithm == "PD":
                denoised_image, self.iter_num = tv_denoise_chambolle(image, self.weight)
                self.process_flag = True
            elif self.algorithm == "FGP":
                denoised_image, self.iter_num = FGP(image, self.weight, self.N)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return None
        else:
            self.algorithm = self.algorithm_combo.currentText()
            image = cv2.imread(self.image_path)
            if self.algorithm == "FGP":
                denoised_image, self.iter_num = FGP(image, self.weight, self.N, channel_axis = True)
                # print(attr)
            #elif self.algorithm == "PD":
                #denoised_image, self.iter_num = tv_denoise_chambolle(image, self.weight, channel_axis = True)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return None
        # return denoised_image, denoised_image_set
    # def display_image(denoised_image):
        # Display the denoised image with new window
        # print the denoised image for 15 seconds
        #TODO：迭代停止出现弹窗
        msgBox = QMessageBox()
        msgBox.setText("Iteration has ended, please view obtained image.");
        msgBox.exec_();
        denoised_image = denoised_image.astype(np.uint8)
        s = denoised_image.shape
        alpha = s[0]/s[1]
        height_const = 400
        #self.image_label.setPixmap(pixmap.scaled(alpha*height_const, height_const))
        cv2.imwrite("denoised_image.jpg", denoised_image)
        pixmap = QPixmap("denoised_image.jpg")
        self.image_label.setPixmap(pixmap.scaled(int(height_const), int(alpha*height_const)))
    def plot_image(self):
        # access iteration time
        assert self.iter_num > 0, "error: no denoising process available."

        # display image for 30 seconds sum(waitkey) = 30000
        num_waitkey = 15000//self.iter_num
        for i in range(self.iter_num+1):
            # Create the path to the temporary image file
            path_temp = "_".join([str(i),"temp.npy"])
            # Check if the temporary image file exists
            if os.path.exists(path_temp):
                print("displaying image", i)
                # Load the image
                deno_temp = np.load(path_temp)
                # display image
                # Convert the image to the appropriate color format
                if self.image_type_combo.currentText() == "B&W":
                    frame = cv2.cvtColor(deno_temp, cv2.COLOR_GRAY2RGB)
                else:
                    frame = deno_temp
                    # frame = cv2.cvtColor(deno_temp, cv2.COLOR_RGB2RGB)
                # Get the height and width of the image
                h, w = deno_temp.shape[:2]
                
                alpha = w/h
                height_const = 400
                h = 400
                w = int(h*alpha)
                # Calculate the bytes per line
                bytesPerLine = 3*w

                # Create a QImage object from the image data
                qimg = QImage(frame.data,w,h,bytesPerLine, QImage.Format.Format_RGB888)
                # Display the frame
                cv2.imshow("Temporary Image", frame)
                # Pause for 0.5 seconds
                cv2.waitKey(num_waitkey)
                # Remove the temporary image file
                os.remove(path_temp)
        cv2.destroyAllWindows()
        # Save the denoised 
    #def save_image(denoised_image):


    # Open a file dialog to download the denoised image
    def download_image(self):
        #TODO: delete npy files if still have
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "No denoised image available.")
            return
        file_dialog = QFileDialog(self)
        file_dialog.setAcceptMode(QFileDialog.AcceptSave)
        file_path, _ = file_dialog.getSaveFileName(self, "Save Denoised Image", "", "Image Files (*.png *.jpg *.jpeg)")
        
        if file_path:
            cv2.imwrite(file_path, cv2.imread("denoised_image.jpg"))
            QMessageBox.information(self, "Success", "Denoised image saved successfully.")

    # Show the application window
    def show(self):
        super().show()
        self.resize(600, 600)

 # Run the application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    image_denoising_app = ImageDenoisingApp()
    image_denoising_app.show()
    sys.exit(app.exec_())